In [9]:
import django
import pandas as pd
import numpy as np
import torch
import logging

django.setup()

logger = logging.getLogger(__name__)

django.setup()

from tqdm.notebook import tqdm

from dbfcore.settings import TZ_UTC
from modelbackend.models import PredictionModel, Signal, PredictionResult, EmbeddedSignal
from modelbackend.management.commands.runmodels import get_datasources_configured_with_env, DataSources
from modelfrontend.utils import get_calc_time_from_timestamp
from modelfrontend.visualization.predictionmodel import create_model_chart, get_history_for_frontend
from modelbackend.management.commands.backfillembeddings import resolve_start_end_timestamps
from modelfrontend.views import backfill_embeddings_for_one_signal, backfill_predictions

torch.set_grad_enabled(False)

#### Input variables setup

In [37]:
start = pd.Timestamp("2024-07-14", tzinfo=TZ_UTC)
end = pd.Timestamp("2024-07-31", tzinfo=TZ_UTC)
forecast_horizon = pd.Timedelta(minutes=60)
model = PredictionModel.objects.get(name="hotmetal_temp_C")
datasouces = get_datasources_configured_with_env()
num_samples = 1000

#### Backfill embeddings for given period

In [ ]:
start_tmstmps, end_tmstmps = resolve_start_end_timestamps(start, end, pd.Timedelta("1W"))
logger.info(f"Calculate embeddings for date range: {start} - {end}")
signalencoders = [i for i in range(1, 26)]

for signalencoder in (pbar := tqdm(signalencoders)):
    pbar.set_description(f"Processing signalencoder {signalencoder}")
    for start_ts, end_ts in (pbar := tqdm(list(zip(start_tmstmps, end_tmstmps)))):
        pbar.set_description(
            f"Processing week {pd.Timestamp.fromtimestamp(start_ts)}-{pd.Timestamp.fromtimestamp(end_ts)}"
        )
        backfill_embeddings_for_one_signal(None, model.pk, signalencoder, start_ts, end_ts, True)

#### Backfill predictions for given period

In [ ]:
start_tmstmps, end_tmstmps = resolve_start_end_timestamps(
    start + pd.Timedelta(days=1), end, pd.Timedelta("1W")
)
logger.info(f"Calculate predictions for date range: {start + pd.Timedelta(days=1)} - {end}")

for start_ts, end_ts in (pbar := tqdm(list(zip(start_tmstmps, end_tmstmps)))):
    pbar.set_description(
        f"Processing week {pd.Timestamp.fromtimestamp(start_ts)}-{pd.Timestamp.fromtimestamp(end_ts)}"
    )
    backfill_predictions(None, model.pk, start_ts, end_ts, True)

#### Evaluation of currently implemented forecast using plotly figure data for one forecast horizon

In [ ]:
def get_target_forecast_times(
    model: PredictionModel, start: pd.Timestamp, end: pd.Timestamp, num_samples: int, datasources: DataSources
) -> pd.Series:
    target = Signal.objects.get(name=model.name)
    targets_df = get_history_for_frontend(target, start, end, end, datasources)[-num_samples:].reset_index()
    targets_df["Timestamp"] = targets_df.apply(
        lambda x: get_calc_time_from_timestamp(x["Timestamp"].timestamp(), model), axis=1
    )
    return targets_df["Timestamp"]


def calculate_forecast_mae(
    model: PredictionModel,
    start: pd.Timestamp,
    end: pd.Timestamp,
    forecast_horizon: pd.Timedelta,
    num_samples: int,
    datasources: DataSources,
):
    forecast_times = get_target_forecast_times(model, start, end, num_samples, datasources)
    calc_times = forecast_times - pd.Timedelta(forecast_horizon)
    abserrors = []
    for calc_time in calc_times:
        fig = create_model_chart(
            model,
            calc_time,
            pd.Timedelta(minutes=0),
            forecast_horizon + pd.Timedelta(minutes=1),
            datasources,
        )
        forecast, target = fig.data[0].y[-1], fig.data[3].y[-1]
        abserror = abs(target - forecast)
        abserrors.append(abserror)

    return np.mean(abserrors)


calculate_forecast_mae(model, start, end, forecast_horizon, num_samples, datasouces)

27.287930202229816